In [244]:
import math
import numpy as np
import random
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import numba
from numba import njit
import random as rd
import time
import graphviz

import plotly.graph_objects as go
import numpy as np

from numba.typed import List
import copy
from scipy.cluster.vq import kmeans2
import matplotlib.pyplot as plt
import numpy as np
from sklearn.cluster import KMeans
from collections import defaultdict

In [245]:
n = 555 # @param {type:"slider", min:1, max:10000, step:1}
a1 = 40 # @param {type:"slider", min:1, max:1000, step:1}
a2 = 60 # @param {type:"slider", min:1, max:1000, step:1}
b1 = 40 # @param {type:"slider", min:1, max:1000, step:1}
b2 = 80 # @param {type:"slider", min:1, max:1000, step:1}
c1 = 40 # @param {type:"slider", min:1, max:1000, step:1}
c2 = 60 # @param {type:"slider", min:1, max:1000, step:1}
item_list = []
for i in range(n):
    a=rd.randint(a1,a2)
    b=rd.randint(b1,b2)
    c=rd.randint(c1,c2)

    item = np.array([a, b, c, 0, 0, 0, 0], np.int_)
    item_list.append(item)
item_list.sort(key= lambda x:(-x[0]+x[1]+x[2]))
item_list= np.array(item_list, np.int_)
bin_length = 500 # @param {type:"slider", min:1, max:10000, step:1}
bin_width = 500 # @param {type:"slider", min:1, max:10000, step:1}
bin_height = 500 # @param {type:"slider", min:1, max:10000, step:1}
bin_size = np.array([bin_length, bin_width, bin_height, 0, 0 ,0], np.int_)
PI_LENGTH = 0
PI_WIDTH = 1
PI_HEIGHT = 2
PI_X = 3
PI_Y = 4
PI_Z = 5
USABLE = 6

In [246]:
@njit
def check(size, pv_x, pv_y, pv_z, max_length, max_width, bin_size):
    if pv_z + size[2] > bin_size[PI_HEIGHT]:
        pv_y += max_width
        pv_z = 0
        size[4] = 1
    if pv_y + size[1] > bin_size[PI_WIDTH]:
        pv_x += max_length
        pv_y = 0
        pv_z = 0
        size[4] = 2
    if pv_x + size[0] > bin_size[PI_LENGTH]:
        size[4] = 3
        return size
    size[3] += abs(max_length - size[0])
    size[3] += abs(max_width - size[1])
    return size

In [247]:
@njit
def shelf(item_list, bin_size):
    pv_x = 0
    pv_y = 0
    pv_z = 0
    max_width = 0
    max_length = 0
    for i in range(len(item_list)):
        a,b,c = item_list[i, PI_LENGTH], item_list[i, PI_WIDTH], item_list[i, PI_HEIGHT]
        arr = np.array([[a,b,c,0,0], [a,c,b,0,0], [b,a,c,0,0], [b,c,a,0,0], [c,a,b,0,0], [c,b,a,0,0]])
        item_size = np.array([0,0,0,100000,0])
        for size in arr:
            size = check(size, pv_x, pv_y, pv_z, max_length, max_width, bin_size)
            if size[4] < item_size[4]:
                item_size = size
                continue
            if size[3] < item_size[3]:
                item_size = size
        item_list[i, PI_LENGTH], item_list[i, PI_WIDTH], item_list[i, PI_HEIGHT] = item_size[0], item_size[1], item_size[2]
        if item_size[4] == 1:
            pv_y += max_width
            pv_z = 0
            max_width = 0
        elif item_size[4] == 2:
            pv_x += max_length
            pv_y = 0
            pv_z = 0
            max_length = 0
        elif item_size[4] == 3:
            continue
        item_list[i][PI_X] = pv_x + bin_size[PI_X]
        item_list[i][PI_Y] = pv_y + bin_size[PI_Y]
        item_list[i][PI_Z] = pv_z + bin_size[PI_Z]
        item_list[i][USABLE] = 1
        pv_z += item_list[i,PI_HEIGHT]
        max_width = max(max_width, item_list[i,PI_WIDTH])
        max_length = max(max_length, item_list[i,PI_LENGTH])
    item_list1=item_list[item_list[:,USABLE]==1]
    item_list2=item_list[item_list[:,USABLE]==0]
    return item_list1, item_list2

In [248]:
@njit
def check_over_lap(x1, a1, y1, b1, x2, a2, y2, b2):
    if x1 >= x2+a2 or x1+a1 <= x2:
        return False
    if y1 >= y2+b2 or y1+b1 <= y2:
        return False
    return True

In [249]:
@njit
def shaking_items(item_list, invert, shaking_direction,bin_size, shaking_zone = np.array([10000000, 10000000, 10000000, 0, 0, 0]), static_zone = np.array([0, 0, 0, -1, -1, -1], np.int_)):
    if not invert:item_list = item_list[np.argsort(item_list[:,PI_LENGTH + shaking_direction] + item_list[:,PI_X + shaking_direction])]
    elif invert: item_list = item_list[np.argsort(-item_list[:,PI_LENGTH + shaking_direction] - item_list[:,PI_X + shaking_direction])]
    for i in range(len(item_list)):
        if item_list[i][PI_X] < shaking_zone[PI_X] or item_list[i][PI_X] > shaking_zone[PI_X] + shaking_zone[PI_LENGTH] or item_list[i][PI_Y] < shaking_zone[PI_Y] or item_list[i][PI_Y] > shaking_zone[PI_Y] + shaking_zone[PI_WIDTH] or item_list[i][PI_Z] < shaking_zone[PI_Z] or item_list[i][PI_Z] > shaking_zone[PI_Z] + shaking_zone[PI_HEIGHT]:
          continue
        if item_list[i][PI_X] >= static_zone[PI_X] and item_list[i][PI_X] <= static_zone[PI_X] + static_zone[PI_LENGTH] and item_list[i][PI_Y] >= static_zone[PI_Y] and item_list[i][PI_Y] <= static_zone[PI_Y] + static_zone[PI_WIDTH] and item_list[i][PI_Z] >= static_zone[PI_Z] and item_list[i][PI_Z] <= static_zone[PI_Z] + static_zone[PI_HEIGHT]:
          continue
        if not invert: new_pos=0
        elif invert:
            new_pos = bin_size[PI_LENGTH + shaking_direction]  - item_list[i][PI_LENGTH + shaking_direction]
        for j in range(i):
            a = (shaking_direction + 1)%3
            b = (shaking_direction + 2)%3
            x = PI_X + a
            y = PI_X + b
            if not invert:
              if check_over_lap(item_list[i, x], item_list[i, a], item_list[i, y], item_list[i, b], item_list[j, x], item_list[j, a], item_list[j, y], item_list[j, b]):
                  new_pos= max(item_list[j][PI_X + shaking_direction]+ item_list[j][PI_LENGTH + shaking_direction], new_pos)
            elif invert:
              if check_over_lap(item_list[i, x], item_list[i, a], item_list[i, y], item_list[i, b], item_list[j, x], item_list[j, a], item_list[j, y], item_list[j, b]):
                  new_pos= min(item_list[j][PI_X + shaking_direction]- item_list[i][PI_LENGTH + shaking_direction], new_pos)
        item_list[i][PI_X + shaking_direction]= new_pos
    return item_list

In [250]:
@njit
def shaking_x(item_list,bin_size, shaking_zone = np.array([10000000, 10000000, 10000000, 0, 0, 0]), static_zone = np.array([0, 0, 0, -1, -1, -1], np.int_)):
    item_list = shaking_items(item_list, 0, 0, bin_size,shaking_zone, static_zone)
    return item_list
@njit
def shaking_x_invert(item_list, bin_size,shaking_zone = np.array([10000000, 10000000, 10000000, 0, 0, 0]), static_zone = np.array([0, 0, 0, -1, -1, -1], np.int_)):
    item_list = shaking_items(item_list, 1, 0, bin_size,shaking_zone, static_zone)
    return item_list
@njit
def shaking_y(item_list,bin_size, shaking_zone = np.array([10000000, 10000000, 10000000, 0, 0, 0]), static_zone = np.array([0, 0, 0, -1, -1, -1], np.int_)):
    item_list = shaking_items(item_list, 0, 1, bin_size,shaking_zone, static_zone)
    return item_list
@njit
def shaking_y_invert(item_list,bin_size, shaking_zone = np.array([10000000, 10000000, 10000000, 0, 0, 0]), static_zone = np.array([0, 0, 0, -1, -1, -1], np.int_)):
    item_list = shaking_items(item_list, 1, 1, bin_size,shaking_zone, static_zone)
    return item_list
@njit
def shaking_z(item_list, bin_size,shaking_zone = np.array([10000000, 10000000, 10000000, 0, 0, 0]), static_zone = np.array([0, 0, 0, -1, -1, -1], np.int_)):
    item_list = shaking_items(item_list, 0, 2, bin_size,shaking_zone, static_zone)
    return item_list
@njit
def shaking_z_invert(item_list,bin_size, shaking_zone = np.array([10000000, 10000000, 10000000, 0, 0, 0]), static_zone = np.array([0, 0, 0, -1, -1, -1], np.int_)):
    item_list = shaking_items(item_list, 1, 2,bin_size, shaking_zone, static_zone)
    return item_list

In [251]:
def draw_packed_bin(item_list, bin_size):
    def get_random_color():
        return "#%06x" % rd.randint(0, 0xFFFFFF)
    color_list  =  []
    for i in range(1000):
        color_list.append(get_random_color())


    data  =  []

    data.append(
        go.Mesh3d(
            x = [0, 0, int(bin_size[0]), int(bin_size[0]), 0, 0, int(bin_size[0]), int(bin_size[0])],
            y = [0, int(bin_size[1]), int(bin_size[1]), 0, 0, int(bin_size[1]), int(bin_size[1]), 0],
            z = [0, 0, 0, 0, int(bin_size[2]), int(bin_size[2]), int(bin_size[2]), int(bin_size[2])],
            name = 'y',
            showscale = True,
            color = "#555",
            opacity = 0.1
        )
    )

    for index in range(len((item_list))):
        item =  item_list[index]
        ox_item  =  int(item[3])
        oy_item  =  int(item[4])
        oz_item  =  int(item[5])

        extreme_x  =  int(item[0]) + ox_item
        extreme_y  =  int(item[1]) + oy_item
        extreme_z  =  int(item[2]) + oz_item

        data.append(
            go.Mesh3d(
                x = [ox_item, ox_item, extreme_x, extreme_x, ox_item, ox_item, extreme_x, extreme_x],
                y = [oy_item, extreme_y, extreme_y, oy_item, oy_item, extreme_y, extreme_y, oy_item],
                z = [oz_item, oz_item, oz_item, oz_item, extreme_z, extreme_z, extreme_z, extreme_z],
                i  =  [7, 0, 0, 0, 4, 4, 6, 6, 4, 0, 3, 2],
                j  =  [3, 4, 1, 2, 5, 6, 5, 2, 0, 1, 6, 3],
                k  =  [0, 7, 2, 3, 6, 7, 1, 1, 5, 5, 7, 6],
                showscale  =  True,
                color  =  get_random_color(),
                hovertext  =  "box "+str(index),
                opacity = 1, flatshading  =  True
            )
        )
        data.append(go.Scatter3d(x = [0], y = [0], z = [0], mode  =  'markers'))

    fig  =  go.Figure(data)
    fig.update_layout(
        autosize = False,
        width = 1300,
        height  =  900,
        scene_aspectmode  =  'data',
        scene_camera  =  dict( eye = dict(x = -2, y = -2, z = -1))
        )

    fig.show()

In [252]:
bin_size_list=[]
k=3
a= bin_size[0]//k
b= bin_size[1]//k
for i in range(k):
    for j in range(k):
        a_bin_size = np.array([a, b, bin_size[2], a*i ,b*j,0], np.int_)

        bin_size_list.append(a_bin_size )

In [253]:
item_list_left = item_list[np.argsort(item_list[:,PI_HEIGHT]**2 + item_list[:, PI_WIDTH] + item_list[:,PI_LENGTH])]
item_list_left
all_item =    np.zeros((0, 7), np.int_)
count = 0
for b in bin_size_list:
    item_list_pack, item_list_left = shelf(item_list_left, b)
    all_item= np.concatenate((all_item,item_list_pack ))
    count += 1
draw_packed_bin(all_item, bin_size)
item_list= all_item

In [254]:
@njit
def check_cost(item_list):
    cost = 0
    for item in item_list:
        cost += item[PI_X]+item[PI_Y]+item[PI_Z]**2
    return cost

In [255]:
@njit
def lap (k, item_list):
    cost = check_cost(item_list)
    for _ in range(1,k):
        l= 100000
        z = rd.randint(1,bin_size[PI_HEIGHT])
        item_list1 = item_list.copy()
        item_list1 = shaking_x_invert(item_list1,bin_size, np.array([l, l, l, 0, 0, z]), np.array([0, 0, 0, -1, -1, -1], np.int_))
        item_list1 = shaking_y_invert(item_list1,bin_size, np.array([l, l, l, 0, 0, z]), np.array([0, 0, 0, -1, -1, -1], np.int_))
        item_list1 = shaking_z(item_list1, bin_size,np.array([l, l, l, 0, 0, z]), np.array([0, 0, 0, -1, -1, -1], np.int_))
        item_list1 = shaking_x(item_list1,bin_size, np.array([l, l, l, 0, 0, z]), np.array([0, 0, 0, -1, -1, -1], np.int_))
        item_list1 = shaking_y(item_list1,bin_size,np.array([l, l, l, 0, 0, z]), np.array([0, 0, 0, -1, -1, -1], np.int_))
        item_list1 = shaking_z(item_list1,bin_size, np.array([l, l, l, 0, 0, z]), np.array([0, 0, 0, -1, -1, -1], np.int_))
        if check_cost(item_list1) < cost:
            cost = check_cost(item_list1)
            item_list = item_list1
    item_list = shaking_x(item_list, bin_size)
    item_list = shaking_y(item_list, bin_size)
    item_list = shaking_z(item_list, bin_size)
    return item_list
print(check_cost(item_list))
item_list = lap(1000, item_list)
print(check_cost(item_list))
draw_packed_bin(item_list, bin_size)

33133932
19061062
